In [12]:
import torch
import nibabel as nib
import os
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset
import numpy as np
import time
from tqdm.auto import tqdm
import utils as utils
from scipy.ndimage import rotate
import matplotlib.pyplot as plt

In [13]:
def get_current_path(path=None):
    if path:
        return path
    else:
        return os.getcwd()

# new_folder = 'data_npy'
# new_folder = 'data_npy2'
new_folder = 'kits23'

if not os.path.exists(f'{get_current_path()}/{new_folder}'):
    os.mkdir(f'{get_current_path()}/{new_folder}')
    
if not os.path.exists(f'{get_current_path()}/{new_folder}/train'):
    os.mkdir(f'{get_current_path()}/{new_folder}/train')
    
if not os.path.exists(f'{get_current_path()}/{new_folder}/valid'):
    os.mkdir(f'{get_current_path()}/{new_folder}/valid')
    
if not os.path.exists(f'{get_current_path()}/{new_folder}/test'):
    os.mkdir(f'{get_current_path()}/{new_folder}/test')
    
def make_path(path):
    if not os.path.exists(path):
        os.makedirs(path)
    return path

train_image_path = make_path(f'{get_current_path()}/{new_folder}/train/image')
train_segment_path = make_path(f'{get_current_path()}/{new_folder}/train/segmentation')
valid_image_path = make_path(f'{get_current_path()}/{new_folder}/valid/image')
valid_segment_path = make_path(f'{get_current_path()}/{new_folder}/valid/segmentation')
test_image_path = make_path(f'{get_current_path()}/{new_folder}/test/image')
test_segment_path = make_path(f'{get_current_path()}/{new_folder}/test/segmentation')

src_path = f'/scratch/student/sinaziaee/datasets/kits23'

In [15]:
def image_slicer(src_path, saving_path, kind='test'):
    count = 0
    depth_list = []
    root_path = os.path.join(src_path, kind)
    for folder_name in tqdm(sorted(os.listdir(root_path))):
        image_path = os.path.join(root_path, folder_name, 'imaging.nii.gz')
        img = nib.load(image_path).get_fdata()
        img=(img-img.min())/(max((img.max()-img.min()),1e-3))
        depth = img.shape[0]
        depth_list.append(depth)
        for j in range(depth):
            new_path=os.path.join(saving_path, '{:05d}.npy'.format(j+count))
            new_img = img[j:j+1, :, :].astype(np.float32)
            np.save(new_path, new_img)  
        count += depth
    return depth_list
depth_list = image_slicer(src_path, saving_path=test_image_path, kind='test')

  0%|          | 0/59 [00:00<?, ?it/s]

In [19]:
import json

depth_lists = {
    'train': [],
    'valid': [],
    'test': depth_list
}

json_file_path = f'{new_folder}/depth_lists.json'

with open(json_file_path, 'w') as json_file:
    json.dump(depth_lists, json_file)
min(depth_list), max(depth_list)

(30, 632)

In [46]:
def segment_slicer(src_path, saving_path, kind='test'):
    count = 0
    depth_list = []
    root_path = os.path.join(src_path, kind)
    for folder_name in tqdm(sorted(os.listdir(root_path))):
        kid1_mask_path = os.path.join(root_path, folder_name, 'instances', 'kidney_instance-1_annotation-1.nii.gz')
        # kid2_mask_path = os.path.join(root_path, folder_name, 'instances', 'kidney_instance-2_annotation-1.nii.gz')
        kid1 = nib.load(kid1_mask_path).get_fdata()
        depth = kid1.shape[0]
        depth_list.append(depth)
        for j in range(depth):
            new_path=os.path.join(saving_path, '{:05d}.npy'.format(j+count))
            seg_1ch = kid1[j:j+1, :, :]
            seg_1ch = torch.tensor(seg_1ch, dtype=torch.int64)
            seg_2ch = F.one_hot(seg_1ch, num_classes=2)
            seg_2ch = torch.squeeze(seg_2ch.permute(3, 0, 1, 2))
            seg_2ch = np.array(seg_2ch, dtype=np.uint8)
            np.save(new_path, seg_2ch)
        count += depth
    return depth_list
depth_list = segment_slicer(src_path, saving_path=test_segment_path, kind='test')

  0%|          | 0/59 [00:00<?, ?it/s]